In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import re
import networkx as nx
import datetime as dt
import time

from cleaning_functions import *
from cleaners import *

%matplotlib notebook

In [ ]:
def get_desc(filename):
    e = ET.parse(filename).getroot()
    for channel in e.iter('channel'):
        print(channel.find('description').text)
    
get_desc('rss_files/joe_rogan.xml')

In [ ]:
df = pd.read_csv('cleaned_podcasts.csv', sep='\t', index_col=0)
df

In [ ]:
split_hosts = splitDataFrameList(df, 'hosts', ', ')
split_hosts

G1 = nx.from_pandas_dataframe(split_hosts, 'guests', 'hosts', edge_attr=['date', 'duration', 'podcast'], create_using=nx.MultiDiGraph())

In [ ]:

G1.edges(data=True)
#nx.average_shortest_path_length(G1)
#nx.eccentricity(G1,'Joe Rogan')
nx.shortest_path_length(G1, "One Pointed Mind")
#nx.shortest_path(G1, 'President Barack Obama', 'Dan Harmon')
#len(G1.edges())

In [ ]:
richard_herring = 'rss_files/richard_herring.xml'

def clean_richard_herring(richard_herring):

    richard_herring = xml_to_df(richard_herring)
        
    date_parser(richard_herring)
    
    richard_herring['guests'] = richard_herring.title.str.extract(r'RHLSTP \d+ - ([\w\s\,\'\`\’\&\"\-]+)',expand=True)

    replace('Trev and Simon', 'Trevor Neal and Simon Hickson', richard_herring)
    
    richard_herring = richard_herring[pd.notnull(richard_herring['guests'])]
    richard_herring = splitDataFrameList(richard_herring, 'guests', ', and ')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ', & ')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ', ')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ',')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ' & ')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ' And ')
    richard_herring = splitDataFrameList(richard_herring, 'guests', ' and ')

    richard_herring = richard_herring[richard_herring.guests.str.contains(r'My ') == False]
    richard_herring = richard_herring[richard_herring.guests.str.contains(r'BLESSED') == False]
    
    richard_herring['guests'] = [g.rstrip() for g in richard_herring['guests']]
    
    return richard_herring

richard_herring = clean_richard_herring(richard_herring)

richard_herring

In [ ]:
slate_money = 'rss_files/slate_money.xml'

def clean_slate_money(slate_money):

    slate_money = xml_to_df_desc(slate_money)
        
    date_parser(slate_money)
    
    slate_money['guests'] = slate_money.description.str.extract(r'(?:with |featuring )([\w\s\,\'\`\’\&\"\-]+).',expand=True)
    slate_money = slate_money.drop('description', 1)

    
    slate_money = slate_money[pd.notnull(slate_money['guests'])]
    slate_money = splitDataFrameList(slate_money, 'guests', ', and ')
    slate_money = splitDataFrameList(slate_money, 'guests', ', & ')
    slate_money = splitDataFrameList(slate_money, 'guests', ', ')
    slate_money = splitDataFrameList(slate_money, 'guests', ',')
    slate_money = splitDataFrameList(slate_money, 'guests', ' & ')
    slate_money = splitDataFrameList(slate_money, 'guests', ' And ')
    slate_money = splitDataFrameList(slate_money, 'guests', ' and ')
    
    guest_split(' of', slate_money)
    guest_split_last("Slate's own ", slate_money)
    guest_split_last("Slate's ", slate_money)

    slate_money['guests'] = [g.rstrip() for g in slate_money['guests']]
    
    return slate_money

slate_money = clean_slate_money(slate_money)

slate_money

In [ ]:
bulletproof_radio = 'rss_files/bulletproof_radio.xml'

def clean_bulletproof_radio(bulletproof_radio):

    bulletproof_radio = xml_to_df(bulletproof_radio)
        
    date_parser(bulletproof_radio)
    
    bulletproof_radio['guests1'] = bulletproof_radio.title.str.extract(r'(?: [Ww]ith | - |: )([^(]+)',expand=True)
    bulletproof_radio['guests2'] = bulletproof_radio.title.str.extract(r'([\w\s\.\,\'\`\’\&\"\-\$]+)(?:: | - )',expand=True)

    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests1'])):
            if(' & ' in row['guests1']):
                bulletproof_radio.at[index, 'guests1'] = ''

    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests2'])):
            if(' & ' in row['guests2']):
                bulletproof_radio.at[index, 'guests2'] = ''
                
    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests1'])):
            if(' on ' in row['guests1']):
                bulletproof_radio.at[index, 'guests1'] = ''

    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests1'])):
            if(' Without ' in row['guests1']):
                bulletproof_radio.at[index, 'guests1'] = ''
                
    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests1'])):
            if(' the ' in row['guests1']):
                bulletproof_radio.at[index, 'guests1'] = ''

    for index, row in bulletproof_radio.iterrows():
        if(pd.notnull(row['guests1'])):
            if('Why ' in row['guests1']):
                bulletproof_radio.at[index, 'guests1'] = ''
                
    bulletproof_radio.replace(np.nan, '', regex=True, inplace=True)
    bulletproof_radio['guests'] = bulletproof_radio['guests1'] + bulletproof_radio['guests2']  
    bulletproof_radio = bulletproof_radio.drop('guests1', 1)
    bulletproof_radio = bulletproof_radio.drop('guests2', 1)
    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    
    guest_split('#', bulletproof_radio)
    guest_split(', Ph.D', bulletproof_radio)
    guest_split(', MD', bulletproof_radio)
    guest_split_last('," ', bulletproof_radio)
    guest_split_last('! ', bulletproof_radio)
    
    guest_split(' - Part', bulletproof_radio)
    guest_split(' Part', bulletproof_radio)
    guest_split(' from ', bulletproof_radio)
    guest_split_last(' - ', bulletproof_radio)
    guest_split_last('with ', bulletproof_radio)
    guest_split_last('w/ ', bulletproof_radio)
    guest_split_last('With ', bulletproof_radio)
    
    guest_split(' -', bulletproof_radio)
    guest_split_last(' – ', bulletproof_radio)
    guest_split_last("uBiome's ", bulletproof_radio)
    guest_split_last("Alderspring's ", bulletproof_radio)
    guest_split_last("Neurosurgeon ", bulletproof_radio)
    
    guest_split('9', bulletproof_radio)
    guest_split('8', bulletproof_radio)
    guest_split('7', bulletproof_radio)
    guest_split('6', bulletproof_radio)
    guest_split('5', bulletproof_radio)
    guest_split('4', bulletproof_radio)
    guest_split('3', bulletproof_radio)
    guest_split('2', bulletproof_radio)
    guest_split('1', bulletproof_radio)
    
    replace('Jayson & Mira Calton', 'Jayson Calton & Mira Calton', bulletproof_radio)
    replace('Mira & Jayson Calton', 'Jayson Calton & Mira Calton', bulletproof_radio)
    replace('Mira and Jayson Calton', 'Jayson Calton & Mira Calton', bulletproof_radio)
    replace('The Bare Minimum From NY Times Best-Selling Author  James Altucher ', 'James Altucher', bulletproof_radio)
    replace("It's Not Liquid, It's Water: ", 'Gerald Pollack', bulletproof_radio)
    replace('Dr. Tami', 'Dr. Tami Meraglia', bulletproof_radio)
    replace('Biohacking Wine: Sulfates, Sugar & Dosing  ', 'Todd White', bulletproof_radio)
    replace('Your Glass Is Always Full Through Irlen Lenses: ', 'Irlen Lenses', bulletproof_radio)
    replace(' Fertility &amp; Food, Flavonoids &amp; Inflammation: ', 'Barry Sears', bulletproof_radio)
    replace('Dr. Mark', 'Dr. Mark Atkinson', bulletproof_radio)
    replace('the Health Ranger', np.nan, bulletproof_radio)
    replace('Alternative Medicine: ', 'Dr. Martin Polanco, Dr. Dan Engle and Deanne Adamson', bulletproof_radio)
    replace('Meat Is The New Ketchup: ', 'Mark Hyman', bulletproof_radio)
    replace('Neurofeedback, Decluttering & Alpha State: ', 'Tony Wrighton', bulletproof_radio)
    replace('Neurogenesis Diet & Combating Cognitive Decline: ', 'Brant Cortright',  bulletproof_radio)
    replace('Entrepreneurship & Alitura Naturals: ', 'Andy Hnilo', bulletproof_radio)
    replace('Shamanism &amp; Energy Fields: ', 'Alberto Villoldo', bulletproof_radio)
    replace('Financial Intelligence &amp; Learning from Failure ', 'Daymond John', bulletproof_radio)
    replace('Bigger, Stronger, Faster', 'Mark Bell, Chris Bell', bulletproof_radio)
    replace('Bulletproof Radio', 'Dr. Terry Wahls', bulletproof_radio)
    replace('Meanwhile, Back at the Ranch', 'Dan Kittredge', bulletproof_radio)
    replace('Food', 'Jonathan Bailor', bulletproof_radio)
    replace('Entrepreneurship & Alitura Naturals', 'Andy Hnilo', bulletproof_radio)
    replace('Dr. Bronner’s Magic', 'Dr. Bronner', bulletproof_radio)
    
    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r' Oil') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Podcast') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Marijuana') == False]
    
    bulletproof_radio = bulletproof_radio[pd.notnull(bulletproof_radio['guests'])]
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', and ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', & ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ',')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' & ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' And ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' and ')
    
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'The ') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Coffee') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Q&A') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Report') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'the Health') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Live ') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r' on ') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r' the ') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'A Second ') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Superman') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Light') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'HRV') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Gratitude') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Health') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Recovery') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Bulletproof') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'founder') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Self') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Beef') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Performance') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Mobility') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'How To') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Hormone') == False]
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r'Plants') == False]
    

    bulletproof_radio['guests'] = [g.rstrip() for g in bulletproof_radio['guests']]
    bulletproof_radio['guests'] = [g.rstrip('.') for g in bulletproof_radio['guests']]

    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    bulletproof_radio = bulletproof_radio[pd.notnull(bulletproof_radio['guests'])]
    
    return bulletproof_radio

clean_bulletproof_radio(bulletproof_radio)

In [ ]:
shane_mauss = 'rss_files/shane_mauss.xml'

def clean_shane_mauss(shane_mauss):

    shane_mauss = xml_to_df_desc(shane_mauss)
    
    shane_mauss['episode'] = len(shane_mauss) - shane_mauss.index
        
    date_parser(shane_mauss)
    
    shane_mauss['guests'] = shane_mauss.description.str.extract(r'with ([\w\s\.\,\'\`\’\&\"\-\‘]+)',expand=True)
    shane_mauss = shane_mauss.drop('description', 1)
    

        
    for index, row in shane_mauss.iterrows():
        if(row['episode']==145):
            shane_mauss.at[index, 'guests'] = 'Eric Osborne'
        if(row['episode']==146):
            shane_mauss.at[index, 'guests'] = 'Emanuel Sferios'
        if(row['episode']==147):
            shane_mauss.at[index, 'guests'] = 'Peter Boghossian'
        if(row['episode']==148):
            shane_mauss.at[index, 'guests'] = 'Katherine MacLean'
        if(row['episode']==149):
            shane_mauss.at[index, 'guests'] = 'Martie Haselton'
        if(row['episode']==150):
            shane_mauss.at[index, 'guests'] = 'Jana Gallus'
        if(row['episode']==151):
            shane_mauss.at[index, 'guests'] = 'Cassie Mogilner Holmes'
        if(row['episode']==152):
            shane_mauss.at[index, 'guests'] = 'Chris Kilham'
        if(row['episode']==153):
            shane_mauss.at[index, 'guests'] = 'James A. Lindsay'
        if(row['episode']==154):
            shane_mauss.at[index, 'guests'] = 'Brittany Alperin'
        if(row['episode']==155):
            shane_mauss.at[index, 'guests'] = 'John Harkness'
        if(row['episode']==158):
            shane_mauss.at[index, 'guests'] = 'Rachel Stone'
        if(row['episode']==159):
            shane_mauss.at[index, 'guests'] = 'Katharine Marske'
        if(row['episode']==160):
            shane_mauss.at[index, 'guests'] = 'Joel Suss'
        if(row['episode']==161):
            shane_mauss.at[index, 'guests'] = np.nan
        if(row['episode']==162):
            shane_mauss.at[index, 'guests'] = np.nan
        if(row['episode']==163):
            shane_mauss.at[index, 'guests'] = 'Rebecca Hood'
            
    shane_mauss = shane_mauss[pd.notnull(shane_mauss['guests'])]
    shane_mauss['guests'] = [g.rstrip('\n \n') for g in shane_mauss['guests']]
    
    guest_split(', Ph.D', shane_mauss)
    guest_split_last('Ph.D. ', shane_mauss)
            
            
    replace('Todd & Viviana Weekes-Shackelford.', 'Todd K. Shackelford & Viviana Weekes-Shackelford.', shane_mauss)
    
    shane_mauss = shane_mauss[pd.notnull(shane_mauss['guests'])]
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ', and ')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ', & ')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ', ')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ',')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ' & ')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ' And ')
    shane_mauss = splitDataFrameList(shane_mauss, 'guests', ' and ')

    shane_mauss = shane_mauss[shane_mauss.guests.str.contains(r'Zookeepers') == False]
    shane_mauss = shane_mauss[shane_mauss.guests.str.contains(r'Live from') == False]

    shane_mauss['guests'] = [g.rstrip('.') for g in shane_mauss['guests']]
    shane_mauss['guests'] = [g.rstrip() for g in shane_mauss['guests']]
    
    return shane_mauss

shane_mauss = clean_shane_mauss(shane_mauss)

shane_mauss

In [2]:
carasantamaria = 'rss_files/carasantamaria.xml'

def clean_carasantamaria(carasantamaria):

    carasantamaria = xml_to_df(carasantamaria)
        
    date_parser(carasantamaria)
    
    carasantamaria['guests'] = carasantamaria.title.str.extract(r' - ([\w\s\,\'\`\’\&\"\-]+)',expand=True)
    
    carasantamaria = carasantamaria[pd.notnull(carasantamaria['guests'])]
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ', and ')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ', & ')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ', ')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ',')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ' & ')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ' And ')
    carasantamaria = splitDataFrameList(carasantamaria, 'guests', ' and ')
    
    carasantamaria = carasantamaria[carasantamaria.guests.str.contains(r'AMA') == False]
    
    guest_split(' 2', carasantamaria)
    guest_split(' 3', carasantamaria)

    carasantamaria['guests'] = [g.rstrip() for g in carasantamaria['guests']]
    
    return carasantamaria

carasantamaria = clean_carasantamaria(carasantamaria)

carasantamaria

,date,duration,guests,title
0,2018-04-02 07:33:32,2711,Melissa Dahl,Episode 201 - Melissa Dahl
2,2018-03-19 20:51:24,5193,Adam Becker,Episode 199 - Adam Becker
3,2018-03-12 07:13:38,3467,Alanna Mitchell,Episode 198 - Alanna Mitchell
4,2018-03-05 21:39:52,5833,David Huntsberger,Episode 197 - David Huntsberger
5,2018-02-27 02:12:12,3656,Laurie Winkless,Episode 196 - Laurie Winkless
6,2018-02-19 21:25:48,4671,Clifford Johnson,Episode 195 - Clifford Johnson
7,2018-02-13 07:16:22,3438,Amanda Hendrix,Episode 194 - Amanda Hendrix
8,2018-02-05 23:02:05,4862,Ian Harris,Episode 193 - Ian Harris
9,2018-01-30 06:15:17,2939,Kelly Loudenberg,Episode 192 - Kelly Loudenberg
10,2018-01-21 07:28:50,4687,James Askew,Episode 191 - James Askew


In [ ]:
econtalk2013 = xml_to_df_desc('rss_files/econtalk2013.xml')
econtalk2006 = xml_to_df_desc('rss_files/econtalk2006.xml')

def clean_econtalk2006(econtalk2006):
        
    date_parser('\s(EST|GMT)$', econtalk2006)
    
    econtalk2006['guests'] = econtalk2006.description.str.extract(r'([\w\s\'\.]+)',expand=True)
    
    guest_split(', ', econtalk2006)
    guest_split(' of', econtalk2006)
    guest_split(' talks', econtalk2006)
    
    guest_split_last('Nobel Laureate ', econtalk2006)
    guest_split_last('syndicated columnist ', econtalk2006)
    guest_split_last('Investigative journalist ', econtalk2006)
    guest_split_last('Author ', econtalk2006)
    guest_split_last('Novelist ', econtalk2006)
    guest_split_last('The legendary ', econtalk2006)
            
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('This is the') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('EconTalk host') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('In this bonus') == False]
    
    econtalk2006 = splitDataFrameList(econtalk2006, 'guests', ' and ')

    return econtalk2006

econtalk2006 = clean_econtalk2006(econtalk2006)

econtalk2006

## podcasts to download

